In [1]:
import pynq
from pynq import Overlay
from pynq.lib.dma import DMA
import numpy as np
from pynq import allocate
import cv2

# Load the overlay that includes the row_parallel_processor module
overlay = Overlay("design_1.bit")  # Ensure you have the correct bitstream file
dma = overlay.axi_dma_0

In [7]:
# Reading image
image_path = "hailee_resized.jpg"  # Path to your image
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
height, width = image.shape
print(f"Image Height: {height}, Width: {width}")

Image Height: 640, Width: 480


In [8]:


# Check if image width is compatible (480 pixels)
if width != 480:
    raise ValueError("Image width must be 480 pixels.")

# Preparing to process the image in rows
rows = height
row_size = 480  # Each row has 480 pixels
chunk_size = row_size  # Each chunk to process is one row

In [9]:


# Allocating buffers for DMA transfer
input_buffer = allocate(shape=(chunk_size,), dtype=np.uint8)  # 8 bits per pixel
output_buffer = allocate(shape=(chunk_size,), dtype=np.uint8)  # 8 bits per pixel

# Initialize output image
processed_image = np.zeros((height, row_size), dtype=np.uint8)  # Grayscale image


In [11]:

# Process each row
for i in range(rows):
    print(f"Processing row: {i}")
    # Copy the row data into input buffer
    np.copyto(input_buffer, image[i, :])  # Copy one row

    # Start DMA transfer
    dma.sendchannel.transfer(input_buffer)
    dma.recvchannel.transfer(output_buffer)

    # Wait for the transfer to complete
    dma.sendchannel.wait()
    dma.recvchannel.wait()

    # Copy processed row back to output image
    processed_image[i, :] = output_buffer

# Save the processed image
cv2.imwrite("processed_image.png", processed_image)
print("Processed image saved as 'processed_image.png'")


Processing row: 0


RuntimeError: DMA channel not idle